In [1]:
%matplotlib inline

In [2]:
from datetime import datetime

In [3]:
#renderer
from PIL import ImageFont
import numpy as np
import cv2
import torch.nn as nn
from torch.nn.utils import clip_grad_norm_

char_size = 24
# char render
def render(text, font=None):
    if font is None:
        font = ImageFont.truetype("/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc", char_size)
    mask = font.getmask(text)
    size = mask.size[::-1]
    a = np.asarray(mask).reshape(size) / 255
    res = cv2.resize(a, dsize=(char_size, char_size), interpolation=cv2.INTER_CUBIC)
    return res

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [5]:
# https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/02-intermediate/language_model/data_utils.py
import torch
import re

class Dictionary(object):
    def __init__(self, max_size=None):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 1
        self.word2idx['⸘'] = 0 # as unk
        self.idx2word[0] = '⸘'
        self.max_size = max_size + 1
    
    def add_word(self, word):
        if not word in self.word2idx and self.idx < self.max_size:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
    
    def __len__(self):
        return len(self.word2idx)


class Corpus(object):
    def __init__(self, max_size=None):
        self.dictionary = Dictionary(max_size=max_size)

    def get_data(self, path, batch_size=20):
        # Add words to the dictionary
#         with open(path, 'r') as f:
#             tokens = 0
#             for line in f:
#                 words = line.split() + ['<eos>']
#                 tokens += len(words)
#                 for word in words: 
#                     self.dictionary.add_word(word)  

        # split words to char and add to dictionary
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                line = ' '.join(line) # split words to char
                line = re.sub(r'[" "]+', ' ', line) # remove continous space
                chars = line.split() + ['¿'] # ¿ as <eos>
                tokens += len(chars)
                for char in chars:
                    self.dictionary.add_word(char)
        
        # Tokenize the file content
        ids = torch.LongTensor(tokens)
        token = 0
        with open(path, 'r') as f:
            for line in f:
                line = ' '.join(line) # split words to char
                line = re.sub(r'[" "]+', ' ', line) # remove continous space
                chars = line.split() + ['¿'] # ¿ as <eos>
                for char in chars:
                    if char in self.dictionary.word2idx:
                        ids[token] = self.dictionary.word2idx[char]
                        token += 1
                    else:
                        ids[token] = self.dictionary.word2idx['⸘']
                        token += 1
        num_batches = ids.size(0) // batch_size
        ids = ids[:num_batches*batch_size]
        return ids.view(batch_size, -1)

In [6]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [7]:
# RNN based language model
class RNNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(RNNLM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.gru = nn.GRU(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, char_id, char_cnn_o, h):
        # Embed word ids to vectors
        x = self.embed(char_id) + char_cnn_o
        
        # Forward propagate LSTM
        out, h = self.gru(x, h)
        
        # Reshape output to (batch_size*sequence_length, hidden_size)
        out = out.reshape(out.size(0)*out.size(1), out.size(2))
        
        # Decode hidden states of all time steps
        out = self.linear(out)
        return out, h

In [8]:
"""
Utility function for computing output of convolutions
takes a tuple of (h,w) and returns a tuple of (h,w)
"""
def conv_output_shape(h_w, kernel_size=1, stride=1, pad=0, dilation=1):
    from math import floor
    if type(kernel_size) is not tuple:
        kernel_size = (kernel_size, kernel_size)
    if type(stride) is not tuple:
        stride = (stride, stride)
    h = floor( ((h_w[0] + (2 * pad) - ( dilation * (kernel_size[0] - 1) ) - 1 )/ stride[0]) + 1)
    w = floor( ((h_w[1] + (2 * pad) - ( dilation * (kernel_size[1] - 1) ) - 1 )/ stride[1]) + 1)
    return h, w

# Dai et al. 's CNN glyph encoder
class Dai_CNN(nn.Module):
    def __init__(self, embed_size, input_size=(32, 32)):
        super(Dai_CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, (7, 7), stride=(2,2))
        torch.nn.init.xavier_uniform_(self.conv1.weight)
        torch.nn.init.zeros_(self.conv1.bias)
        h, w = conv_output_shape(input_size, (7, 7), (2, 2))
        
        self.conv2 = nn.Conv2d(32, 16, (5, 5), stride=(2,2))
        torch.nn.init.xavier_uniform_(self.conv2.weight)
        torch.nn.init.zeros_(self.conv2.bias)
        h, w = conv_output_shape((h, w), (5, 5), (2, 2))
                
        self.fc = nn.Linear(16*h*w, embed_size)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        torch.nn.init.zeros_(self.fc.bias)
        
        self.h, self.w = h, w
        
    def forward(self, char_img):
        b = char_img.size(0)
        x = self.conv1(char_img)
        x = torch.nn.functional.relu(x)
        x = self.conv2(x)
        x = torch.nn.functional.relu(x)
        x = x.view(-1, 16*self.h*self.w)
        return self.fc(x)

In [9]:
class ImageClassification(nn.Module):
    def __init__(self, hidden_size, classes):
        super(ImageClassification, self).__init__()
        self.hidden = nn.Linear(hidden_size, classes)
    
    def forward(self, cnn_o):
        return self.hidden(cnn_o)

In [10]:
# Hyper-parameters
embed_size = 300
hidden_size = 128
num_layers = 1
num_epochs = 50
batch_size = 16
seq_length = 32
learning_rate = 1e-3

# Load dataset
corpus = Corpus(max_size=4000)
ids = corpus.get_data('icwb2-data/training/msr_training.utf8', batch_size)
vocab_size = len(corpus.dictionary)
num_batches = ids.size(1) // seq_length

In [11]:
# save char images for reference
for char, idx in corpus.dictionary.word2idx.items():
    np.save(f'char_img/noto_CJK/msr/{idx}.npy', render(char))

In [12]:
model = RNNLM(vocab_size, embed_size, hidden_size, num_layers).to(device)
cnn_encoder = Dai_CNN(embed_size, (char_size, char_size)).to(device)
clf = ImageClassification(embed_size, vocab_size).to(device)
model.train()
cnn_encoder.train()
clf.train()
params = list(model.parameters())+list(cnn_encoder.parameters()) + list(clf.parameters())

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params, lr=learning_rate)

# Truncated backpropagation
def detach(state):
    return state.detach()

In [13]:
# Train the model
for epoch in range(num_epochs):
    global_step = 0 
    glyph_warmup = 0
    glyph_ratio = 0.1
    glyph_decay = 0.1
    # Set initial hidden and cell states
    state = torch.zeros(num_layers, batch_size, hidden_size).to(device)
    
    for i in range(0, ids.size(1) - seq_length, seq_length):
        # Get mini-batch inputs and targets
        inputs = ids[:, i:i+seq_length].to(device)
        targets = ids[:, (i+1):(i+1)+seq_length].to(device)
        
        # Get images
        images = np.zeros((inputs.size(0), inputs.size(1), char_size, char_size, 1))
        for b, seq in enumerate(ids):
            for s, idx in enumerate(ids[b][i:i+seq_length]):
                images[b, s] = np.load(f'char_img/noto_CJK/msr/{idx}.npy').reshape(char_size,char_size,1)
        images = torch.from_numpy(images).float().to(device) # B N H W C
        images = images.view(-1, char_size, char_size, 1) # B*N H W C
        images = images.permute(0, 3, 1, 2) # from B*N H W C to B*N C H W
        
        # Get encoded images
        cnn_o = cnn_encoder(images)
        
        # Forward pass of Classification Task
        clf_o = clf(cnn_o)
        loss_clf = criterion(clf_o, targets.reshape(-1))
                      
        # Forward pass of RNNLM
        cnn_o = cnn_o.view(inputs.size(0), inputs.size(1), -1)
        state = detach(state)
        outputs, state = model(inputs, cnn_o, state)
        loss_lm = criterion(outputs, targets.reshape(-1))
        
        if global_step < glyph_warmup:
            loss = (1 - glyph_ratio) * loss_lm + glyph_ratio * loss_clf
        else:
            loss_ratio = glyph_ratio * glyph_decay ** (i + 1 + global_step // 5)
            loss = (1 - loss_ratio) * loss_lm + loss_ratio * loss_clf
        
        # Backward and optimize
        model.zero_grad()
        cnn_encoder.zero_grad()
        loss.backward()
        clip_grad_norm_(params, 0.5)
        optimizer.step()
        global_step += 1

        step = (i+1) // seq_length
        if step % 1000 == 0:
            print ('{} Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
                   .format(datetime.now(), epoch+1, num_epochs, step, num_batches, loss.item(), np.exp(loss.item())))

2019-11-27 18:12:21.153890 Epoch [1/50], Step[0/8080], Loss: 8.3136, Perplexity: 4078.90
2019-11-27 18:15:18.997133 Epoch [1/50], Step[1000/8080], Loss: 5.0811, Perplexity: 160.96
2019-11-27 18:18:17.908673 Epoch [1/50], Step[2000/8080], Loss: 4.6229, Perplexity: 101.79
2019-11-27 18:21:15.196696 Epoch [1/50], Step[3000/8080], Loss: 4.4064, Perplexity: 81.98
2019-11-27 18:24:13.361497 Epoch [1/50], Step[4000/8080], Loss: 4.2224, Perplexity: 68.20
2019-11-27 18:27:06.658774 Epoch [1/50], Step[5000/8080], Loss: 4.6678, Perplexity: 106.46
2019-11-27 18:29:59.347630 Epoch [1/50], Step[6000/8080], Loss: 4.6531, Perplexity: 104.91
2019-11-27 18:32:52.768386 Epoch [1/50], Step[7000/8080], Loss: 4.1191, Perplexity: 61.51
2019-11-27 18:35:41.835422 Epoch [1/50], Step[8000/8080], Loss: 4.3034, Perplexity: 73.95
2019-11-27 18:35:55.704732 Epoch [2/50], Step[0/8080], Loss: 4.2709, Perplexity: 71.58
2019-11-27 18:38:41.992344 Epoch [2/50], Step[1000/8080], Loss: 4.1539, Perplexity: 63.68
2019-11-27

2019-11-27 22:10:04.335536 Epoch [11/50], Step[2000/8080], Loss: 3.8856, Perplexity: 48.70
2019-11-27 22:12:52.465489 Epoch [11/50], Step[3000/8080], Loss: 3.7704, Perplexity: 43.40
2019-11-27 22:15:41.765332 Epoch [11/50], Step[4000/8080], Loss: 3.7356, Perplexity: 41.91
2019-11-27 22:18:30.128154 Epoch [11/50], Step[5000/8080], Loss: 4.2791, Perplexity: 72.18
2019-11-27 22:21:20.376426 Epoch [11/50], Step[6000/8080], Loss: 3.9506, Perplexity: 51.97
2019-11-27 22:24:06.080005 Epoch [11/50], Step[7000/8080], Loss: 3.6808, Perplexity: 39.68
2019-11-27 22:26:54.139484 Epoch [11/50], Step[8000/8080], Loss: 3.9245, Perplexity: 50.63
2019-11-27 22:27:07.374179 Epoch [12/50], Step[0/8080], Loss: 3.8492, Perplexity: 46.95
2019-11-27 22:29:56.402528 Epoch [12/50], Step[1000/8080], Loss: 3.6561, Perplexity: 38.71
2019-11-27 22:32:43.151774 Epoch [12/50], Step[2000/8080], Loss: 3.8737, Perplexity: 48.12
2019-11-27 22:35:32.760514 Epoch [12/50], Step[3000/8080], Loss: 3.7576, Perplexity: 42.84
20

2019-11-28 02:00:27.454595 Epoch [21/50], Step[3000/8080], Loss: 3.6987, Perplexity: 40.40
2019-11-28 02:03:16.511480 Epoch [21/50], Step[4000/8080], Loss: 3.6543, Perplexity: 38.64
2019-11-28 02:06:06.200249 Epoch [21/50], Step[5000/8080], Loss: 4.2183, Perplexity: 67.92
2019-11-28 02:08:55.838235 Epoch [21/50], Step[6000/8080], Loss: 3.8776, Perplexity: 48.31
2019-11-28 02:11:42.669574 Epoch [21/50], Step[7000/8080], Loss: 3.6566, Perplexity: 38.73
2019-11-28 02:14:30.806274 Epoch [21/50], Step[8000/8080], Loss: 3.9125, Perplexity: 50.02
2019-11-28 02:14:44.338418 Epoch [22/50], Step[0/8080], Loss: 3.8214, Perplexity: 45.67
2019-11-28 02:17:33.287894 Epoch [22/50], Step[1000/8080], Loss: 3.6511, Perplexity: 38.52
2019-11-28 02:20:22.926488 Epoch [22/50], Step[2000/8080], Loss: 3.8379, Perplexity: 46.43
2019-11-28 02:23:14.178356 Epoch [22/50], Step[3000/8080], Loss: 3.7279, Perplexity: 41.59
2019-11-28 02:26:04.720046 Epoch [22/50], Step[4000/8080], Loss: 3.6438, Perplexity: 38.24
20

2019-11-28 05:50:47.819285 Epoch [31/50], Step[4000/8080], Loss: 3.6332, Perplexity: 37.83
2019-11-28 05:53:37.346235 Epoch [31/50], Step[5000/8080], Loss: 4.1803, Perplexity: 65.38
2019-11-28 05:56:25.587306 Epoch [31/50], Step[6000/8080], Loss: 3.8362, Perplexity: 46.35
2019-11-28 05:59:15.551742 Epoch [31/50], Step[7000/8080], Loss: 3.6280, Perplexity: 37.64
2019-11-28 06:02:05.515473 Epoch [31/50], Step[8000/8080], Loss: 3.9082, Perplexity: 49.81
2019-11-28 06:02:19.152231 Epoch [32/50], Step[0/8080], Loss: 3.8338, Perplexity: 46.24
2019-11-28 06:05:05.011440 Epoch [32/50], Step[1000/8080], Loss: 3.6028, Perplexity: 36.70
2019-11-28 06:07:55.254631 Epoch [32/50], Step[2000/8080], Loss: 3.8580, Perplexity: 47.37
2019-11-28 06:10:46.133183 Epoch [32/50], Step[3000/8080], Loss: 3.6912, Perplexity: 40.09
2019-11-28 06:13:35.133753 Epoch [32/50], Step[4000/8080], Loss: 3.6500, Perplexity: 38.48
2019-11-28 06:16:26.970008 Epoch [32/50], Step[5000/8080], Loss: 4.1808, Perplexity: 65.42
20

2019-11-28 09:41:06.942738 Epoch [41/50], Step[5000/8080], Loss: 4.1936, Perplexity: 66.26
2019-11-28 09:43:53.651293 Epoch [41/50], Step[6000/8080], Loss: 3.8024, Perplexity: 44.81
2019-11-28 09:46:41.664878 Epoch [41/50], Step[7000/8080], Loss: 3.6004, Perplexity: 36.61
2019-11-28 09:49:27.925591 Epoch [41/50], Step[8000/8080], Loss: 3.9094, Perplexity: 49.87
2019-11-28 09:49:41.395565 Epoch [42/50], Step[0/8080], Loss: 3.8209, Perplexity: 45.65
2019-11-28 09:52:30.326263 Epoch [42/50], Step[1000/8080], Loss: 3.6155, Perplexity: 37.17
2019-11-28 09:55:19.303795 Epoch [42/50], Step[2000/8080], Loss: 3.8507, Perplexity: 47.03
2019-11-28 09:58:05.324105 Epoch [42/50], Step[3000/8080], Loss: 3.6726, Perplexity: 39.35
2019-11-28 10:00:54.998469 Epoch [42/50], Step[4000/8080], Loss: 3.6442, Perplexity: 38.25
2019-11-28 10:03:45.506824 Epoch [42/50], Step[5000/8080], Loss: 4.1835, Perplexity: 65.60
2019-11-28 10:06:31.240613 Epoch [42/50], Step[6000/8080], Loss: 3.7984, Perplexity: 44.63
20

In [15]:
model.eval()
cnn_encoder.eval()
clf.eval()
total_clf_loss = .0
perplexity = .0
num_step = 0
for i in range(0, ids.size(1) - seq_length, seq_length):
    # Get mini-batch inputs and targets
    inputs = ids[:, i:i+seq_length].to(device)
    targets = ids[:, (i+1):(i+1)+seq_length].to(device)

    # Get images
    images = np.zeros((inputs.size(0), inputs.size(1), char_size, char_size, 1))
    for b, seq in enumerate(ids):
        for s, idx in enumerate(ids[b][i:i+seq_length]):
            images[b, s] = np.load(f'char_img/noto_CJK/msr/{idx}.npy').reshape(char_size,char_size,1)
    images = torch.from_numpy(images).float().to(device) # B N H W C
    images = images.view(-1, char_size, char_size, 1) # B*N H W C
    images = images.permute(0, 3, 1, 2) # from B*N H W C to B*N C H W

    # Forward pass of Classification Task
    cnn_o = cnn_encoder(images)
    clf_o = clf(cnn_o)
    loss_clf = criterion(clf_o, targets.reshape(-1))
    
    total_clf_loss += loss_clf.item()
    
    # Get encoded images
    cnn_o = torch.reshape(cnn_o, (inputs.size(0), inputs.size(1), -1))

    # Forward pass
    state = detach(state)
    outputs, state = model(inputs, cnn_o, state)
    loss = criterion(outputs, targets.reshape(-1))
    
    perplexity += np.exp(loss.item())

    num_step += 1
    
print(f"Train Perplexity: {perplexity / num_step}")
print(f"Classification loss: {total_clf_loss / num_step}")

Train Perplexity: 44.1288733378575
Classification loss: 17.111882503729056


In [16]:
test_ids = corpus.get_data('icwb2-data/testing/msr_test.utf8', batch_size)
# filter out unknown character
test_ids = test_ids.view(-1)
mask = test_ids < vocab_size
test_ids = test_ids[mask]
num_batches = test_ids.size(0) // batch_size
test_ids = test_ids[:num_batches*batch_size]
test_ids = test_ids.view(batch_size, -1)

In [17]:
perplexity = .0
num_step = 0
total_clf_loss = .0
for i in range(0, test_ids.size(1) - seq_length, seq_length):
    # Get mini-batch inputs and targets
    inputs = test_ids[:, i:i+seq_length].to(device)
    targets = test_ids[:, (i+1):(i+1)+seq_length].to(device)

    # Get images
    images = np.zeros((inputs.size(0), inputs.size(1), char_size, char_size, 1))
    for b, seq in enumerate(test_ids):
        for s, idx in enumerate(test_ids[b][i:i+seq_length]):
            images[b, s] = np.load(f'char_img/noto_CJK/msr/{idx}.npy').reshape(char_size,char_size,1)
    images = torch.from_numpy(images).float().to(device) # B N H W C
    images = images.view(-1, char_size, char_size, 1) # B*N H W C
    images = images.permute(0, 3, 1, 2) # from B*N H W C to B*N C H W
    
    # Forward pass of Classification Task
    cnn_o = cnn_encoder(images)
    clf_o = clf(cnn_o)
    loss_clf = criterion(clf_o, targets.reshape(-1))
    
    total_clf_loss += loss_clf.item()

    # Get encoded images
    cnn_o = torch.reshape(cnn_o, (inputs.size(0), inputs.size(1), -1))

    # Forward pass
    state = detach(state)
    outputs, state = model(inputs, cnn_o, state)
    loss = criterion(outputs, targets.reshape(-1))

    perplexity += np.exp(loss.item())

    num_step += 1
    
print(f"Test Perplexity: {perplexity / num_step}")
print(f"Classification loss: {total_clf_loss / num_step}")

Test Perplexity: 58.066234992868445
Classification loss: 16.723388851176164
